In [ ]:
import os
import openai
import backoff
import time
import json
import pdb
import argparse
import traceback
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

from utils import *

openai.api_key = ""
STOP_PHRASE = "END"

##### refer to the paper, "Human-like Summarization Evaluation with ChatGPT"

# Likert scale scroing example

In [ ]:
article = "LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in \"Harry Potter and the Order of the Phoenix\" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. \"I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,\" he told an Australian interviewer earlier this month. \"I don't think I'll be particularly extravagant. \"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs.\" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film \"Hostel: Part II,\" currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. \"I'll definitely have some sort of party,\" he said in an interview. \"Hopefully none of you will be reading about it.\" Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. \"People are always looking to say 'kid star goes off the rails,'\" he told reporters last month. \"But I try very hard not to go that way because it would be too easy for them.\" His latest outing as the boy wizard in \"Harry Potter and the Order of the Phoenix\" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called \"My Boy Jack,\" about author Rudyard Kipling and his son, due for release later this year. He will also appear in \"December Boys,\" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's \"Equus.\" Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: \"I just think I'm going to be more sort of fair game,\" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."
summary = "Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund ."
prompt = "Evaluate the quality of summaries written for a news article. Rate each summary on four dimensions: relevance, informativeness, fluency, and coherence. You should rate on a scale from 1 (worst) to 5 (best)."

query = f"{prompt}\nArticle: {article}\nSummary: {summary}"
batch = [{"role": "user", "content": f"{query}"}]
# print(batch[0]['content'])
response = create_completion(model=get_model_id('gpt-3.5-turbo-0613'), messages=batch, max_tokens=256,
                                            temperature=0, n=1, stop=STOP_PHRASE,model_type='chat',
                                            log_path=os.path.join('./logs', 'log.txt'))

print(response['choices'])

# Pairwise comparison

In [ ]:
article = "LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in \"Harry Potter and the Order of the Phoenix\" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. \"I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,\" he told an Australian interviewer earlier this month. \"I don't think I'll be particularly extravagant. \"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs.\" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film \"Hostel: Part II,\" currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. \"I'll definitely have some sort of party,\" he said in an interview. \"Hopefully none of you will be reading about it.\" Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. \"People are always looking to say 'kid star goes off the rails,'\" he told reporters last month. \"But I try very hard not to go that way because it would be too easy for them.\" His latest outing as the boy wizard in \"Harry Potter and the Order of the Phoenix\" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called \"My Boy Jack,\" about author Rudyard Kipling and his son, due for release later this year. He will also appear in \"December Boys,\" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's \"Equus.\" Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: \"I just think I'm going to be more sort of fair game,\" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."
summary1 = "Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund ."
summary2 = "Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away ."
prompt = "Given a new article, which summary is better? Answer \"Summary 0\" or \"Summary 1\". You do not need to explain the reason."

query = f"{prompt}\nArticle: {article}\nSummary0: {summary1}\nSummary1: {summary2}"
batch = [{"role": "user", "content": f"{query}"}]
# print(batch[0]['content'])
response = create_completion(model=get_model_id('gpt-3.5-turbo-0613'), messages=batch, max_tokens=256,
                                            temperature=0, n=1, stop=STOP_PHRASE,model_type='chat',
                                            log_path=os.path.join('./logs', 'log.txt'))

print(response['choices'])

# How to make input files

In [ ]:
import pandas as pd
import json

task_name = "likert"
model_name = "free"

prompt_likert = "Evaluate the quality of summaries written for a news article. Rate each summary on four dimensions: relevance, informativeness, fluency, and coherence. You should rate on a scale from 1 (worst) to 5 (best)."
prompt_comp = "Given a new article, which summary is better? Answer \"Summary 0\" or \"Summary 1\". You do not need to explain the reason."

TASK = {"likert": prompt_likert, "comp": prompt_comp}
prompt = TASK[task_name]

df_article = pd.read_csv("./data/context.csv", index_col=False)
df_free = pd.read_csv("./data/predict_free.csv", index_col=False)
df_full = pd.read_csv("./data/predict_full.csv", index_col=False)

jobs = []
for i in range(len(df_article)):
    article = df_article.iloc[i, 1]
        
    if task_name == "likert":
        summary = df_free.iloc[i, 1] if model_name == 'free' else df_full.iloc[i, 1]
        query = f"{prompt}\nArticle: {article}\nSummary: {summary}"
    elif task_name == "comp":
        model_name = "free_full"
        summary0, summary1 = df_free.iloc[i, 1], df_full.iloc[i, 1]
        query = f"{prompt}\nArticle: {article}\nSummary0: {summary0}\nSummary1: {summary1}"
    
    jobs.append({"model": "gpt-3.5-turbo-0613", "n": 1, "temperature": 0.0, "messages": [{"role": "user", "content": f'{query}'}]})
    
with open(f"./inputs/{task_name}_{model_name}.jsonl", "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

# How to get results

In [ ]:
task_name = "comp"
model_name = "full_free"

results = ()
with open(f"./outputs/{task_name}_{model_name}.jsonl", "r") as f:
    for line in f:
        if line[3] == '[':
            results += (eval(line[4:-2])[1],)
        elif line[4] == '[':
            results += (eval(line[5:-2])[1],)

if task_name == "likert":
    scores = {'Relevance': 0, 'Informativeness': 0, 'Fluency': 0, 'Coherence': 0}
    num = 100

    for res in results:
        values = res['choices'][0]['message']['content']
        try:
            for cat in values.split('\n'):
                key, v = cat.split(':')
                scores[key] += int(v)
        except:
            # print(values.split('\n'))
            num -= 1
        
    avg_score = [round(v / num, 2) for _, v in scores.items()]
    print(avg_score, num)
    
elif task_name == 'comp':
    scores = {'Summary0': 0, 'Summary1': 0}
    
    for res in results:
        values = res['choices'][0]['message']['content']
        
        if 'Summary0' in values or 'Summary 0' in values: scores['Summary0'] += 1
        elif 'Summary1' in values or 'Summary 1' in values: scores['Summary1'] += 1
        else: print(values)
        
    print(scores)